# 특정 거리의 도시 찾기
boj 18352

- Solve idea
    - 최단거리가 나와있으므로 BFS로 문제를 풀 예정이다.
    - 노드랑 노드 길이를 같이 넣어서(최단 길이 이므로) 풀어준다

In [ ]:
import sys
import queue
N,M,K,X = map(int,sys.stdin.readline().split())
node = [[] for _ in range(0,N+1)]
visited = [0]*(N+1)
for _ in range(M):
    a,b = map(int,sys.stdin.readline().split())
    node[a].append(b)


#시작
q = queue.Queue()
result = []
length = 1
for i in node[X]:
    q.put((i,length))
visited[X] = 1
while not q.empty():
    x, node_length = q.get()
    if node_length == K and not visited[x]:
        result.append(x)
        visited[x] = 1
        continue
    if not visited[x]:
        visited[x] = 1
        for ix in node[x]:
            q.put((ix,node_length+1))

if result:
    result.sort()
    for i in result:
        print(i)
else:
    print(-1)

# 연구소
boj 14502


- Solve idea
    - node에 모든 경우로 3개 벽 박기
      - 백트래킹
    - dfs로 바이러스 전이
      - dfs
    - 0인 부분 dfs로 개수 계산
      - 배열에서 0인 부분 찾기
    - 반복
copy보다 그냥 순수 배열 복사가 더 빠른 듯 싶다.

### 나의 풀이

In [ ]:
import sys
import copy
N,M = map(int,sys.stdin.readline().split())
node = [list(map(int,sys.stdin.readline().split())) for _ in range(N)]
result = 0
new_node = [[0]*(M) for _ in range(N)]
d = [(1,0),(-1,0),(0,1),(0,-1)]

# 벽 3개 박는 코드 ->바이러스(2) 찾고 dfs 함수 실행
def dfs(X,Y):
    for ik in d:
        x,y = ik
        nx, ny = X+x, Y+y
        if 0<=nx<N and 0<=ny<M and not new_node[nx][ny]:# 0이면
            new_node[nx][ny] = 2
            dfs(nx,ny)
def wall(cnt):
    global result
    global new_node
    
    if cnt == 3:
        # 새 배열로 복사
        temp_node = copy.deepcopy(node)
        new_node = temp_node
        for ix in range(N):
            for jx in range(M):
                if new_node[ix][jx] ==2: # 바이러스이고 방문하지 않았으면 바이러스 퍼뜨림
                    dfs(ix,jx)
                    
        # 0인 구역 크기 계산
        cnts = 0
        for iv in range(N):
            for jv in range(M):
                if not new_node[iv][jv]: #0이면
                    cnts += 1
        # 큰 값 비교
        result = max(result,cnts)
        return 
    for i in range(N):
        for j in range(M):
            if not node[i][j]: # 0이면
                node[i][j] = 1
                wall(cnt+1) # 벽 세우고
                node[i][j] = 0 # 다시 허뭄
wall(0)
print(result)

### 정답

In [ ]:
# BOJ에서는 [언어]를 PyPy3로 설정하여 제출해주세요.

n, m = map(int, input().split())
data = [] # 초기 맵 리스트
temp = [[0] * m for _ in range(n)] # 벽을 설치한 뒤의 맵 리스트

for _ in range(n):
    data.append(list(map(int, input().split())))

# 4가지 이동 방향에 대한 리스트
dx = [-1, 0, 1, 0]
dy = [0, 1, 0, -1]

result = 0

# 깊이 우선 탐색(DFS)을 이용해 각 바이러스가 사방으로 퍼지도록 하기
def virus(x, y):
    for i in range(4):
        nx = x + dx[i]
        ny = y + dy[i]
        # 상, 하, 좌, 우 중에서 바이러스가 퍼질 수 있는 경우
        if nx >= 0 and nx < n and ny >= 0 and ny < m:
            if temp[nx][ny] == 0:
                # 해당 위치에 바이러스 배치하고, 다시 재귀적으로 수행
                temp[nx][ny] = 2
                virus(nx, ny)

# 현재 맵에서 안전 영역의 크기 계산하는 메서드
def get_score():
    score = 0
    for i in range(n):
        for j in range(m):
            if temp[i][j] == 0:
                score += 1
    return score

# 깊이 우선 탐색(DFS)을 이용해 울타리를 설치하면서, 매 번 안전 영역의 크기 계산
def dfs(count):
    global result
    # 울타리가 3개 설치된 경우
    if count == 3:
        for i in range(n):
            for j in range(m):
                temp[i][j] = data[i][j]
        # 각 바이러스의 위치에서 전파 진행
        for i in range(n):
            for j in range(m):
                if temp[i][j] == 2:
                    virus(i, j)
        # 안전 영역의 최대값 계산
        result = max(result, get_score())
        return
    # 빈 공간에 울타리를 설치
    for i in range(n):
        for j in range(m):
            if data[i][j] == 0:
                data[i][j] = 1
                count += 1
                dfs(count)
                data[i][j] = 0
                count -= 1

dfs(0)
print(result)

# 경쟁적 전염
boj 18405

- Solve idea
  - K개의 배열을 만들고 virus배열을 돌면서 1번,2번... 바이러스의 좌표를 넣는다.
  - 그 좌표로 bfs로 돈다 -> 1초 추가
  - 해당 초가 될 때 까지 반복

In [ ]:
import sys
import queue
input = sys.stdin.readline
N, K = map(int,input().split())

virus, temp = [], []
for i in range(N):
    virus.append(list(map(int,input().split())))
    for j in range(N):
        if virus[i][j]:
            temp.append((virus[i][j],i,j))
            
S, X, Y = map(int,input().split())
d = [(1,0),(-1,0),(0,1),(0,-1)]

def bfs(li,X,Y):
    q = queue.Queue()
    # queue 배열 생성
    for s,xs,xy in li:
        q.put((xs,xy))
    cnt = 0 
    while (not q.empty()) and (cnt != S):
        for _ in range(q.qsize()):
            x, y = q.get()
            for ix,iy in d:
                nx, ny = x+ix, y+iy
                if (0<=nx<N) and (0<=ny<N) and (not virus[nx][ny]):
                    virus[nx][ny] = virus[x][y]
                    q.put((nx,ny))
        
        cnt += 1
    return virus[X-1][Y-1]

temp.sort()
print(bfs(temp,X,Y))

# 괄호 변환
- Solve idea
    1. 문자열 p가 **올바른 문자열**인지 체크
        - 올바른 문자열이면 그대로 return 아니라면 그 다음 단계
    2. u,v 분리
       - u는 '가장 작은 균형잡힌 괄호 문자열' v는 나머지
    3. u가 **올바른 문자열**인지 체크
       - 맞을 시
         - 나머지에 대해 된다.
       - 아닐 시
         - 새로운 빈 문자열에 '('
         - v에 대해 1단계부터 다시 (재귀)
         - ')' 붙이기
         - u의 첫, 마지막 문자 제거 및 괄호 방향 다 뒤집기.
         - 다 합친거 return

In [11]:
def alright_checker(s):
    temp = []
    if s[0] == ')': # 올바른 문자가 아님
        return False
        
    for i in s:
        if i == '(':
            temp.append(i)
        else:
            # early stopping
            if not temp: # temp가 비어 있으면 올바른 문자가 아님
                return False
            prev = temp[-1]
            # )일 때
            if prev=='(':
                temp.pop()
            else:
                temp.append(i)
    if not temp:
        return True
    else:
        return False

In [28]:
def split(s):
    left, right = 0,0
    for i in range(len(s)):
        if s[i] ==')':
            right += 1
        else:
            left += 1
        if right == left:
            print(i, right, left)
            return s[:i+1], s[i+1:] # return u, v  

In [30]:
def solution(p):
    if not p: # step 0 (빈 문자열)
        return p
    if alright_checker(p): # step 1
        return p
    v = p
    u, v = split(v) # step 2
    if alright_checker(u): # step 3-1 (올바른 문자열이 맞을 시)
        return u + solution(v)
    else: # step 3-2 (올바른 문자열이 아닐 시)
        left = '('+ solution(v) + ')'
        right = ''
        for i in u[1:-1]:
            if i == ')':
                right += '('
            else:
                right += ')'
        return left + right

# 연산자 끼워넣기
- Solve idea
  - 연산자를 리스트로 변환
  - 그 리스트 순열 조합으로 변환
  - 순열 조합을 돌면서 연산
  - 나눗셈 처리 (문제 대로)

```
# case 1
per_arr = list(permutations(opers, N-1))
for oper in per_arr:

# case 2
for oper in permutations(opers, N-1):
```
- case 1보다 2가 더 빠름
  - list로 변환하는 작업이 리소스를 잡아먹는다


In [ ]:
import sys
from itertools import permutations
input = sys.stdin.readline

N = int(input())
arr = list(map(int,input().split()))
operator = list(map(int,input().split()))

opers =[]
for i in range(len(operator)):
    if i == 0:
        op = '+'
    elif i == 1:
        op = '-'
    elif i == 2:
        op = '*'
    else:
        op = '/'
    for i in range(operator[i]):
        opers.append(op)
        

def custom_integer_division(dividend, divisor):
    # 음수를 양수로 바꾼 뒤 몫을 취하고, 그 몫을 다시 음수로 바꿔줌
    if dividend < 0 and divisor > 0 or dividend > 0 and divisor < 0:
        return -(-dividend // abs(divisor))
    else:
        return dividend // divisor
        
result_max, result_min = -1234567890, 1234567890
per_arr = list(permutations(opers, N-1))
for oper in per_arr:
    result_num = arr[0]
    for j in range(len(oper)):
        if oper[j] == '+':
            result_num += arr[j+1]
        elif oper[j] == '-':
            result_num -= arr[j+1]
        elif oper[j] == '/':
            result_num = custom_integer_division(result_num, arr[j+1])

        else:
            result_num *= arr[j+1]
    result_max = max(result_max, result_num)
    result_min = min(result_min, result_num)
    
print(result_max)
print(result_min)

# 감시 피하기
- Solve idea
  - input을 받을 때 선생님 위치 정보를 같이 받는다.
  - 그게 아니라면 3개의 장애물을 놓는다. (NC3 (조합))
  - 3개의 장애물을 놓고 bfs다시 수행
  - 감시를 피하면 Yes, 아니면 No

In [ ]:
import sys
import queue
input = sys.stdin.readline

N = int(input())
# input
arr, corr = [], []
for i in range(N):
    arr.append(list(input().rstrip().split()))
    for j in range(N):
        if arr[i][j] == 'T':
            corr.append((i,j))


d = [(1,0),(-1,0),(0,1),(0,-1)]
temp = [[0]*(N) for _ in range(N)] # 빈 배열
result =  False

def bfs(arr):
    # 새 배열에 추가
    for i in range(N):
        for j in range(N):
            temp[i][j] = arr[i][j]
            
    # 선생님 좌표 queue에 넣기 (변하지 않음)
    for ix,iy in corr:
        for dx,dy in d:
            q = queue.Queue()
            q.put((ix,iy))
            while not q.empty():
                x, y = q.get()
                nx, ny = x+dx, y+dy
                if 0<=nx<N and 0<=ny<N:
                    if temp[nx][ny]=='S':
                        return False
                    elif temp[nx][ny]=='O': # 학생, 장애물 일 경우
                        break
                    else:
                        q.put((nx,ny))
    return True

def obstacle(cnt):
    global result
    if cnt == 3:
        cnt = 0
        if bfs(arr) and not result: # early stop
            result = True
            return 
    else:
        for i in range(N):
            for j in range(N):
                if arr[i][j]=='X':
                    arr[i][j]='O'
                    cnt += 1
                    obstacle(cnt)
                    arr[i][j]='X'
                    cnt -= 1

obstacle(0)
if result:
    print("YES")
else:
    print("NO")

# 인구이동
boj 16234

- Solve idea
  - 모든 나라 각각 bfs로 상하좌우를 돌며 차이를 계산한다. (절대값으로)
  - 차이가 L~R에 든다면 연합 인구에 추가한다. 인덱스도 넣어야 할 듯?(무조건 하나라도 성립 시)
  - 다 돌면 연합 인구 계산하고, 저장된 인덱스에 추가하면서 연합 인구가 없을 때 까지 돈다.
## Queue 를 사용했더니 시간이 터짐
- deque를 사용했더니 된다.
- Queue는 thread-safe한 큐를 제공하므로 싱글 스레드 환경에서는 deque가 조금 빠를 수 있습니다.

In [ ]:
import sys
from collections import deque

input = sys.stdin.readline
N, L, R = map(int,input().split())
population = [list(map(int,input().split())) for _ in range(N)]
d = [(1,0),(-1,0),(0,1),(0,-1)]
def bfs(x,y):
    q = deque()
    union = []
    
    q.append((x,y))
    union.append((x,y))
    while q:
        ox,oy = q.popleft()
        for ix, iy in d:
            nx, ny = ox + ix, oy + iy
            if 0<=nx<N and 0<=ny<N and not visited[nx][ny] and L<= abs(population[ox][oy] - population[nx][ny]) <=R:
                visited[nx][ny] = 1
                q.append((nx,ny))
                union.append((nx,ny))
    return union
    
cnt = 0
while True:
    flag = 0
    visited = [[0]*(N) for _ in range(N)]
    for i in range(N):
        for j in range(N):
            if not visited[i][j]:
                visited[i][j] = 1
                unions = bfs(i,j)
                if len(unions) > 1:
                    flag = 1
                    num = sum([population[ix][iy] for ix, iy in unions]) // len(unions)
                    for dx,dy in unions:
                        population[dx][dy] = num
                        
    if not flag: #연합 국가 없음. 끝
        break
    cnt += 1
print(cnt)

In [19]:
int((50+30+30) / 3)

36

In [16]:
population

[[35, 35], [35, 35]]

In [20]:
s = [[4, 4, 4, 4, 4, 4, 4, 4, 4], [4, 4, 4, 4, 4, 4, 4, 4, 4], [4, 4, 4, 4, 4, 4, 4, 4, 4], [4, 4, 4, 4, 4, 4, 4, 4, 4], [4, 4, 4, 4, 4, 4, 4, 4, 4], [4, 4, 4, 4, 4, 4, 4, 4, 4], [1, 1, 1, 4, 4, 4, 4, 4, 4], [4, 4, 4, 4, 4, 4, 4, 4, 4], [4, 4, 4, 4, 4, 4, 4, 4, 4]]

In [21]:
for i in s:
    for j in i:
        print(j, end=' ')
    print()

4 4 4 4 4 4 4 4 4 
4 4 4 4 4 4 4 4 4 
4 4 4 4 4 4 4 4 4 
4 4 4 4 4 4 4 4 4 
4 4 4 4 4 4 4 4 4 
4 4 4 4 4 4 4 4 4 
1 1 1 4 4 4 4 4 4 
4 4 4 4 4 4 4 4 4 
4 4 4 4 4 4 4 4 4 
